# E-Commerce MCP Server - Strands Agent Demo

This notebook demonstrates how to connect a Strands Agent to the E-Commerce MCP Server using streamable HTTP transport with authentication.

## 1. Install Dependencies

Install required packages: `strands-agents`, `mcp`, and `requests`.

In [ ]:
!pip install -r requirements.txt

## 2. Authenticate and Connect to MCP Server

Get a JWT token via `/auth/login`, then create an authenticated MCP client.

In [ ]:
import os
import requests
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands.tools.mcp import MCPClient

# MCP Server URL - update this to your deployed server
MCP_SERVER_URL = os.getenv("MCP_SERVER_URL", "http://your-server:8000")

# Step 1: Authenticate and get JWT token
response = requests.post(
    f"{MCP_SERVER_URL}/auth/login",
    json={"email": "demo2@example.com", "password": "Demo123!"}
)
token = response.json()["access_token"]

# Step 2: Create MCP client with Bearer token authentication
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        url=f"{MCP_SERVER_URL}/mcp",
        headers={"Authorization": f"Bearer {token}"}
    )
)

print("MCP client ready!")

## 3. Test Connection (Optional)

Quick test to verify the MCP server is reachable and list available tools.

In [ ]:
with mcp_client:
    # List available tools
    tools = mcp_client.list_tools_sync()
    print(f"\nAvailable tools ({len(tools)}):")
    for tool in tools:
        # MCPAgentTool uses tool_name attribute
        name = getattr(tool, 'tool_name', None) or getattr(tool, 'name', str(tool))
        print(f"  - {name}")

    print("\nConnection successful!")


## 4. Multi-Turn Chat with Strands Agent

**Important:** The agent must be used INSIDE the `with mcp_client:` context manager. The MCP connection is only active within this context.

In [ ]:
# Multi-turn chat - agent MUST be inside the context manager
with mcp_client:
    # Load tools and create agent inside the context
    tools = mcp_client.list_tools_sync()
    print(f"Loaded {len(tools)} MCP tools")
    
    agent = Agent(
        tools=tools,
        system_prompt= """You are a helpful shopping assistant. Be concise.
  You are already authenticated - don't ask users for login credentials.
  Just proceed with orders, reviews, and other authenticated operations."""
    )
    
    print("Type 'quit' to exit\n")
    print("-" * 60)
    
    # Multi-turn conversation loop - stays inside context
    while True:
        user_input = input("\n👤 User: ")
        if user_input.strip().lower() == "quit":
            print("👋 Goodbye!")
            break
        
        response = agent(user_input)
        print(f"\n🤖 Agent: {response}")
        print("-" * 60)